In [2]:
from pathlib import Path
import json

class Environment() :
    def __init__(self, exchange=0, path=None, dataSize=72, dataType='h', bonusData=12) :
        self.path = Path('data' if path==None else path)
        
        if isinstance(exchange, int) :
            exchanges = [ex for ex in self.path.glob('*') if ex.is_dir()]
            self.exchange = exchanges[exchange]
        elif isinstance(exchange, str) :
            self.exchange = self.path / exchange
            if not self.exchange.is_dir() : 
                raise ValueError("Exchange not found")

        self.dataSize = dataSize
        self.dataType = dataType
        self.load()
        
    def load(self) :
        
        data = json.loads(file.read_text())

        pass
    
    def getCurrencies(self) :
        pass
    
    def reset(self) :
        pass
    
    def readData(self) :
        pass
    
    def eval(self) :
        pass


In [3]:
#import CryptEnv

c = CryptEnv.Environment(exchange='Kraken')

print(c.path, c.exchange)


data data\Kraken
